In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
# Import mlc_data.csv
mlc_df = pd.read_csv("../data/mlc_data.csv")
mlc_df.head()

,If,Country Code,Registrant Code,Year of Reference,Usage Period,Streaming Platform (DSP),Streams,Recording Artist,Recording Label,Recording Title,Release Title,Recording Duration (Seconds),"Songwriter's Listed (1 = ""Yes"", 0 = ""No"")"
0,USUG12101043,US,UG1,21.0,NaN,AudioMack,"10,175",Future,NaN,FUTURE FT DEJ LOAF HEY THERE PROD BY DDS,#unknown#,181,0
1,USRC12100543,US,RC1,21.0,NaN,SoundCloud,"8,597",LUCKY3RD,LUCKY3RD,Keep It Cool LUCKY3RD,Keep It Cool LUCKY3RD,133,0
2,USSM12102263,US,SM1,21.0,NaN,SoundCloud,"261,280",LUCKY3RD,LUCKY3RD,Life Goes On LUCKY3RD,Life Goes On LUCKY3RD,171,0
3,USLD91731547,US,LD9,17.0,2/1/2021,Trebel,5,Bachata & Merengue Mix,Orchard,No dudes de mi- Merengue & Bachata Mix,Mega Mix 2010,1250,0
4,USAT22007048,US,AT2,20.0,NaN,AudioMack,"62,105",Foolio,NaN,WHEN I SEE YOU REMIX,#unknown#,187,0


In [3]:
# Abi: Rename column If to ISRC -- ORIGINAL 'If ' column HAS A SPACE after it!!
mlc_df.columns = ['ISRC', 'Country', 'Registrant', 'Release_Year', 'Period', 'DSP', 'Streams', 'Artist', 'Label', 'Song', 'Album', 'Duration', 'Songwriter?']
mlc_df.head()

,ISRC,Country,Registrant,Release_Year,Period,DSP,Streams,Artist,Label,Song,Album,Duration,Songwriter?
0,USUG12101043,US,UG1,21.0,NaN,AudioMack,"10,175",Future,NaN,FUTURE FT DEJ LOAF HEY THERE PROD BY DDS,#unknown#,181,0
1,USRC12100543,US,RC1,21.0,NaN,SoundCloud,"8,597",LUCKY3RD,LUCKY3RD,Keep It Cool LUCKY3RD,Keep It Cool LUCKY3RD,133,0
2,USSM12102263,US,SM1,21.0,NaN,SoundCloud,"261,280",LUCKY3RD,LUCKY3RD,Life Goes On LUCKY3RD,Life Goes On LUCKY3RD,171,0
3,USLD91731547,US,LD9,17.0,2/1/2021,Trebel,5,Bachata & Merengue Mix,Orchard,No dudes de mi- Merengue & Bachata Mix,Mega Mix 2010,1250,0
4,USAT22007048,US,AT2,20.0,NaN,AudioMack,"62,105",Foolio,NaN,WHEN I SEE YOU REMIX,#unknown#,187,0


In [4]:
mlc_df.tail()

,ISRC,Country,Registrant,Release_Year,Period,DSP,Streams,Artist,Label,Song,Album,Duration,Songwriter?
99995,QZK6F2019397,QZ,K6F,20.0,3/1/2021,Spotify,"28,549",Myuk,Sony Music Labels Inc.,魔法 - Anime Size,魔法 (Anime Size),89,1
99996,QZK6F2019397,QZ,K6F,20.0,3/1/2021,Spotify,"28,941",yama,Sony Music Labels Inc.,麻痺,麻痺,199,1
99997,SE6HN1926755,SE,6HN,19.0,3/1/2021,Spotify,"28,941",yama,Sony Music Labels Inc.,麻痺,麻痺,199,1
99998,SEYOK1669274,SE,YOK,16.0,3/1/2021,Apple,"8,970",yama,Sony Music Labels Inc.,麻痺,麻痺 - Single,198,1
99999,QZK6F2019397,QZ,K6F,20.0,2/1/2021,Spotify,"11,245",Lexie Liu,泥鞋（北京）文化传媒有限公司,黑洞 Metropolis,无限意识 Meta Ego,259,1


In [5]:
# Inspecting the dataframe made from mlc_data.csv
print(mlc_df.shape)
print(mlc_df.dtypes)

(100000, 13)
ISRC             object
Country          object
Registrant       object
Release_Year    float64
Period           object
DSP              object
Streams          object
Artist           object
Label            object
Song             object
Album            object
Duration          int64
Songwriter?       int64
dtype: object


In [6]:
# Most popular streaming platform by number of records in the dataframe
mlc_df['DSP'].value_counts()

Spotify               32268
Apple                 22200
Amazon                14438
Pandora               13777
Tidal                  3521
YouTube                2752
SoundCloud             2122
GTL                    1090
Melodyv                 829
Trebel                  817
iHeart Radio            707
AudioMack               550
NugsNet                 316
LiveXLive               200
Qoboz                   104
Midwest Tape            102
Deezer                   51
Anghami                  46
Sonos                    23
Recisio                  22
Smithsonian              21
Ultimate Guitar          19
PowerMusic                8
Wolfgangs                 4
Fan Label                 4
MixCloud                  4
Pacemaker                 3
Classical Archives        2
MonkingMe                 1
Name: DSP, dtype: int64

In [7]:
# Abi:
# mlc_df["Streams"] = mlc_df["Streams"].str.replace(",", "")
# mlc_df["Streams"] = mlc_df["Streams"].str.replace(" ", "")
# mlc_df[["Streams"]] = mlc_df[["Streams"]].apply(pd.to_numeric)

In [8]:
# Patrick: Most popular streaming platform by number of records in the dataframe with US country code
# domestic = mlc_df.loc[mlc_df['Country']== 'US']
# domestic['DSP'].value_counts()

In [9]:
# Patrick: Most popular streaming platform by number of records in the dataframe with non-US (foreign) country code
# foreign = mlc_df.loc[mlc_df['Country']!= 'US']
# foreign['DSP'].value_counts()

In [10]:
# Most popular streams of all
# mlc_df.sort_values('Streams')

In [11]:
# Rudy: More info
mlc_df.describe()

,Release_Year,Duration,Songwriter?
count,98239.000000,100000.000000,100000.000000
mean,19.671933,1016.893690,0.541610
std,12.638870,15565.692133,0.498268
min,0.000000,0.000000,0.000000
25%,17.000000,149.000000,0.000000
50%,19.000000,190.000000,1.000000
75%,20.000000,236.000000,1.000000
max,99.000000,818738.000000,1.000000


## Finding the ratio (=0-1) of songwriter info provided by DSP

In [12]:
# Exploring Songwriter? values -- there are more recordings that have songwriter info provided than not
mlc_df['Songwriter?'].value_counts()

1    54161
0    45839
Name: Songwriter?, dtype: int64

In [13]:
# Rank of DSPs by songwriter ratio
sw_ratio = mlc_df.groupby('DSP')['Songwriter?'].mean()
sw_ratio.sort_values(ascending=False)

DSP
Classical Archives    1.000000
Qoboz                 0.903846
Deezer                0.862745
Spotify               0.854376
Apple                 0.808514
Recisio               0.545455
Amazon                0.261740
Pandora               0.244828
Melodyv               0.114596
LiveXLive             0.070000
GTL                   0.028440
Tidal                 0.003692
Smithsonian           0.000000
Ultimate Guitar       0.000000
Trebel                0.000000
Wolfgangs             0.000000
SoundCloud            0.000000
Sonos                 0.000000
YouTube               0.000000
Pacemaker             0.000000
PowerMusic            0.000000
Anghami               0.000000
NugsNet               0.000000
MonkingMe             0.000000
MixCloud              0.000000
Midwest Tape          0.000000
Fan Label             0.000000
AudioMack             0.000000
iHeart Radio          0.000000
Name: Songwriter?, dtype: float64

## We can make a bar plot of the top 6 vs. bottom 6 that have a non-zero ratio